# Bird-beak outcomes
@Author Max Frohlich<br>
@Date 5 Aug 2019<br>

# Purpose
To measure the effect of pre-operative geometry on post-operative bird-beaking severity.

# Scope
Uses parameters measured from MATLAB code and SimVascular. Specific parameters used as data inputs:
1. BBH (post-op)
2. BBL (post-op)
3. BBA (post-op)
4. Graft Diameter (post-op, from spec sheet)
5. Aortic Diameter (pre-op, from SimVascular)
6. Aortic Area (pre-op, from MATLAB)
7. Aortic Curvature (pre-op, from MATLAB)


In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from functools import reduce
import warnings
warnings.filterwarnings('ignore')

# Read in data

In [40]:
bb_data = pd.read_csv('../data/bbdata_area.csv')
bb_data.set_index('pat_id',inplace=True)

# Analysis method

In [36]:
class BirdBeakOutcomes:
    """
    This class analyzes pre-operative geometry to determine if their is any relationship with post-operative
    bird-beaking severity. 
    
        Attributes:
        -----------
        df [dataframe]
            The patient-level result-data used for analysis. 
            Columns:
            --------
            bbh (float)
                Bird-beak height (mm)
            bbl (float)
                Bird-beak length (mm)
            bba (float)
                Bird-beak angle (degrees)
            graft (int)
                Graft diameter (mm)
            area (float)
                Aortic area (mm2)
            curve (float)
                Curvature at proximal landing point (mm-1)
            effective_diameter (float)
                Diameter of aorta at proximal landing point (mm)
            percent_oversizing (float)
                Ratio of graft diameter to aortic diameter
            CD (float)
                Element-wise multiplication of curve and effective_diameter
            
                
            
        """
    
    def __init__(self, df):
        
        self.df = df.copy()

    def define_bird_threshold(self, threshold_mm):
        """ 
        Use this function to define what bird-beaking is using bbh as the variable
        BBG = Bird-beak group
        NBBG = No-Bird-beak group
        """
        self.df['group'] = self.df['bbh'].apply(lambda x: 'BBG' if x >= threshold_mm else 'NBBG')
        self.threshold_mm = threshold_mm
    
    def agg_results(self):
        """ 
        Aggregates the population into mean and standard deviation.
        Assumes 3 populations: BBG, NBBG, and All (BBG + NBBG)
        
        """
        # calculate metrics
        self._calc_diameter()
        self._calc_oversizing()
        self._calc_curvature_diameter_product()        
        
        # format column names
        self._format_output_colnames()
        
        # Make a copy of the original dataframe so we can measure all
        self.df_agg = self.df.copy()
        self.df_agg['group'] = 'All'
        self.df_agg = pd.concat([self.df, self.df_agg])
        self.df_agg = self.df_agg.groupby('group', as_index=False).agg([np.mean, np.std])    
        
    def calc_pvals(self):
        """ Performs independent t-test on BBG and NBBG"""
        for feat in self.cols:
            bbgroup = self.df[self.df['group'] == 'BBG']
            nobbgroup = self.df[self.df['group'] == 'NBBG']
            s, p = stats.ttest_ind(bbgroup[feat], nobbgroup[feat])
            self.df_agg.loc['p-value', feat] = ['{:.3f}'.format(p)]
        self.df_agg.fillna('',inplace=True)
        self.df_agg = self.df_agg.transpose()
        self.df_agg = self.df_agg.reindex(sorted(self.df_agg.index.values))
        self.df_agg.style.applymap(self.bold_significance, subset='p-value')    
        
    def _calc_diameter(self):
        """ Calculate effective diameter from area"""
        self.df['effective_diameter'] = self.df['area'].apply(lambda x: np.sqrt((4 * x)/np.pi))
        
    def _calc_oversizing(self):
        """ Calculate percent oversizing"""
        self.df['percent_oversizing'] = 100*((self.df['graft'] / self.df['effective_diameter'])-1)
        
    def _calc_curvature_diameter_product(self):
        """ Curvature * diameter metric"""
        self.df['CD'] = self.df['effective_diameter'] * self.df['curve']
        
    def _format_output_colnames(self):
        """ Formats colnames so output will look cleaner """
        self.df.rename(columns={
                                'bbh': 'BBH (mm)', 
                                'curve':'Curvature (mm-1)', 
                                'effective_diameter': 'Diameter (mm)', 
                                'percent_oversizing':'Graft Oversizing (%)',
                                'bba': 'BBA (deg)',
                                'bbl': 'BBL (mm)',
                                'graft': 'Proximal Graft Diameter (mm)',
                                'area': 'Aortic Area (mm2)'
                                }, inplace=True)    
        

    def format_for_output_display(self):
        """ Formats table into something that's easier to copy-paste into Excel or Word"""
        one_sigfig = ['BBL (mm)', 'BBH (mm)','Diameter (mm)','CD']
        three_sigfig = ['Curvature (mm-1)']
        results_temp = []
        
        self.cols = ['Proximal Graft Diameter (mm)',
                'Aortic Area (mm2)',
                'Curvature (mm-1)', 
                'Diameter (mm)', 
                'CD', 
                'Graft Oversizing (%)',
                'BBL (mm)', 
                'BBH (mm)', 
                'BBA (deg)']
        
        for col in self.cols:
            fstring = "{0:.3f} ± {1:.3f}" if col in three_sigfig else ("{0:.1f} ± {1:.1f}" if col in one_sigfig else "{0:.0f} ± {1:.0f}")
            mean_std = self.df_agg[col]
            mean_std['mean ± std'] = mean_std.apply(lambda x: fstring.format(x['mean'], x['std']),axis=1)
            mean_std.drop(columns=['mean', 'std'],inplace=True)
            mean_std.index = ['All', 'BBG', 'NBBG']
            mean_std.columns = pd.MultiIndex.from_product([[col], mean_std.columns])
            results_temp.append(mean_std)
        self.df_agg = reduce(lambda left, right: pd.merge(left, right, left_index=True, right_index=True),results_temp)
    
    @staticmethod
    def bold_significance(val):
        """ Bold significant values"""
        if type(val)==str:
            val = float(val.replace('<', ''))
        fontweight = 'bold' if val<0.05 else 'normal'
        return 'font-weight: {}'.format(fontweight)
    
    def display_results(self):
        """ Displays output results"""
        try:
            bbg_n = len(self.df[self.df['group']=='BBG'])
            nbbg_n = len(self.df[self.df['group']=='NBBG'])
            print('Bird-beak threshold of {} mm'.format(self.threshold_mm))
            print('BBG vs NBBG N: {} vs {}'.format(bbg_n, nbbg_n))
            display(self.df_agg.style.applymap(self.bold_significance, subset='p-value'))
        except:
            display(self.df_agg)

# Helper function to call a range of thresholds

In [46]:
def perform_outcomes(df, threshold):
    birdbeak_obj = BirdBeakOutcomes(bb_data)
    birdbeak_obj.define_bird_threshold(threshold)
    birdbeak_obj.agg_results()
    birdbeak_obj.format_for_output_display()
    birdbeak_obj.calc_pvals()
    birdbeak_obj.display_results()

In [48]:
perform_outcomes(bb_data, threshold=3)
perform_outcomes(bb_data, threshold=4)
perform_outcomes(bb_data, threshold=5)
perform_outcomes(bb_data, threshold=6)
perform_outcomes(bb_data, threshold=7)

Bird-beak threshold of 3 mm
BBG vs NBBG N: 14 vs 6


,,All,BBG,NBBG,p-value
Aortic Area (mm2),mean ± std,918 ± 208,969 ± 152,800 ± 283,0.098
BBA (deg),mean ± std,24 ± 10,28 ± 7,14 ± 8,0.002
BBH (mm),mean ± std,4.7 ± 2.8,6.1 ± 2.0,1.5 ± 0.5,0.000
BBL (mm),mean ± std,5.0 ± 4.5,6.9 ± 4.1,0.6 ± 0.2,0.002
CD,mean ± std,1.2 ± 0.5,1.3 ± 0.5,0.9 ± 0.2,0.041
Curvature (mm-1),mean ± std,0.035 ± 0.013,0.038 ± 0.014,0.029 ± 0.010,0.156
Diameter (mm),mean ± std,34.0 ± 4.1,35.0 ± 2.8,31.5 ± 5.7,0.077
Graft Oversizing (%),mean ± std,9 ± 8,8 ± 8,11 ± 7,0.440
Proximal Graft Diameter (mm),mean ± std,37 ± 5,38 ± 4,35 ± 6,0.246


Bird-beak threshold of 4 mm
BBG vs NBBG N: 12 vs 8


,,All,BBG,NBBG,p-value
Aortic Area (mm2),mean ± std,918 ± 208,988 ± 152,813 ± 244,0.062
BBA (deg),mean ± std,24 ± 10,27 ± 8,19 ± 11,0.057
BBH (mm),mean ± std,4.7 ± 2.8,6.6 ± 1.8,1.9 ± 0.9,0.000
BBL (mm),mean ± std,5.0 ± 4.5,8.0 ± 3.4,0.6 ± 0.2,0.000
CD,mean ± std,1.2 ± 0.5,1.3 ± 0.5,1.0 ± 0.4,0.101
Curvature (mm-1),mean ± std,0.035 ± 0.013,0.038 ± 0.014,0.032 ± 0.013,0.342
Diameter (mm),mean ± std,34.0 ± 4.1,35.4 ± 2.8,31.8 ± 5.0,0.056
Graft Oversizing (%),mean ± std,9 ± 8,8 ± 9,10 ± 6,0.515
Proximal Graft Diameter (mm),mean ± std,37 ± 5,38 ± 4,35 ± 6,0.181


Bird-beak threshold of 5 mm
BBG vs NBBG N: 10 vs 10


,,All,BBG,NBBG,p-value
Aortic Area (mm2),mean ± std,918 ± 208,1002 ± 118,834 ± 248,0.068
BBA (deg),mean ± std,24 ± 10,28 ± 8,20 ± 10,0.065
BBH (mm),mean ± std,4.7 ± 2.8,7.0 ± 1.7,2.5 ± 1.4,0.000
BBL (mm),mean ± std,5.0 ± 4.5,8.8 ± 3.2,1.3 ± 1.5,0.000
CD,mean ± std,1.2 ± 0.5,1.4 ± 0.5,1.0 ± 0.3,0.030
Curvature (mm-1),mean ± std,0.035 ± 0.013,0.040 ± 0.014,0.031 ± 0.012,0.133
Diameter (mm),mean ± std,34.0 ± 4.1,35.7 ± 2.1,32.2 ± 4.9,0.059
Graft Oversizing (%),mean ± std,9 ± 8,7 ± 10,10 ± 6,0.352
Proximal Graft Diameter (mm),mean ± std,37 ± 5,38 ± 4,36 ± 6,0.273


Bird-beak threshold of 6 mm
BBG vs NBBG N: 5 vs 15


,,All,BBG,NBBG,p-value
Aortic Area (mm2),mean ± std,918 ± 208,985 ± 71,896 ± 234,0.422
BBA (deg),mean ± std,24 ± 10,31 ± 11,21 ± 8,0.044
BBH (mm),mean ± std,4.7 ± 2.8,8.4 ± 1.0,3.5 ± 1.9,0.000
BBL (mm),mean ± std,5.0 ± 4.5,11.1 ± 2.9,3.0 ± 2.8,0.000
CD,mean ± std,1.2 ± 0.5,1.7 ± 0.6,1.0 ± 0.3,0.004
Curvature (mm-1),mean ± std,0.035 ± 0.013,0.048 ± 0.017,0.031 ± 0.010,0.013
Diameter (mm),mean ± std,34.0 ± 4.1,35.4 ± 1.3,33.5 ± 4.6,0.380
Graft Oversizing (%),mean ± std,9 ± 8,5 ± 12,10 ± 6,0.235
Proximal Graft Diameter (mm),mean ± std,37 ± 5,37 ± 3,37 ± 6,0.940


Bird-beak threshold of 7 mm
BBG vs NBBG N: 4 vs 16


,,All,BBG,NBBG,p-value
Aortic Area (mm2),mean ± std,918 ± 208,967 ± 68,906 ± 230,0.612
BBA (deg),mean ± std,24 ± 10,34 ± 11,21 ± 8,0.020
BBH (mm),mean ± std,4.7 ± 2.8,8.8 ± 0.3,3.7 ± 2.0,0.000
BBL (mm),mean ± std,5.0 ± 4.5,11.4 ± 3.2,3.4 ± 3.2,0.000
CD,mean ± std,1.2 ± 0.5,1.6 ± 0.7,1.1 ± 0.4,0.053
Curvature (mm-1),mean ± std,0.035 ± 0.013,0.045 ± 0.018,0.033 ± 0.011,0.091
Diameter (mm),mean ± std,34.0 ± 4.1,35.1 ± 1.2,33.7 ± 4.5,0.557
Graft Oversizing (%),mean ± std,9 ± 8,8 ± 12,9 ± 7,0.842
Proximal Graft Diameter (mm),mean ± std,37 ± 5,38 ± 3,37 ± 5,0.697
